# pγ→ηπ⁰p with SymPy only

This section is a follow-up to formulate the amplitude model for the $\gamma p \to \eta\pi^0 p$ channel example symbolically. See **[TR&#8209;033](https://compwa.github.io/report/033)** for a purely numerical tutorial.

The model we want to implement is

$$
\begin{array}{rcl}
I &=& \left|A^{12} + A^{23} + A^{31}\right|^2 \\
A^{12} &=& \frac{\sum a_m Y_2^m (\Omega_1)}{s_{12}-m^2_{a_2}+im_{a_2} \Gamma_{a_2}} \\
A^{23} &=& \frac{\sum b_m Y_1^m (\Omega_2)}{s_{23}-m^2_{\Delta}+im_{\Delta} \Gamma_{\Delta}} \\
A^{31} &=& \frac{c_0}{s_{31}-m^2_{N^*}+im_{N^*} \Gamma_{N^*}} \,,
\end{array}
$$

where $1=\eta$, $2=\pi^0$, and $3=p$.

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import sympy as sp

## Model implementation

### $A^{12}$

In [ ]:
s12, m_a2, Gamma_a2 = sp.symbols(r"s_{12} m_{a_2} \Gamma_{a_2}")
theta1, phi1 = sp.symbols("theta_1 phi_1")
a = sp.IndexedBase("a")
m = sp.symbols("m", cls=sp.Idx)
A12 = sp.Sum(a[m] * sp.Ynm(2, m, theta1, phi1), (m, -2, 2)) / (
    s12 - m_a2**2 + sp.I * m_a2 * Gamma_a2
)
A12

In [ ]:
sp.Ynm(2, 1, 1, 1).expand(func=False)

In [ ]:
sp.Ynm(2, 1, 1, 1).expand(func=True)

In [ ]:
A12.free_symbols

In [ ]:
A12_func = sp.lambdify(
    [s12, a[-2], a[-1], a[0], a[1], a[2], m_a2, Gamma_a2, theta1, phi1],
    A12.doit().expand(func=True),
)
A12_func

### $A^{23}$

In [ ]:
s23, m_delta, gamma_delta = sp.symbols("s_{23} m_Delta Gamma_Delta")
b = sp.IndexedBase("b")
m = sp.symbols("m", cls=sp.Idx)

theta2, phi2 = sp.symbols("theta_2 phi_2")
A23 = sp.Sum(b[m] * sp.Ynm(1, m, theta2, phi2), (m, -1, 1)) / (
    s23 - m_delta**2 + sp.I * m_delta * gamma_delta
)
A23

In [ ]:
A23_func = sp.lambdify(
    [s23, b[-1], b[0], b[1], m_delta, gamma_delta, theta2, phi2],
    A23.doit().expand(func=True),
)

### $A^{31}$

In [ ]:
s31, c0, m_nstar, gamma_nstar = sp.symbols(r"s_{31} c_0 m_{N^*} \Gamma_{N^*}")
A31 = c0 / (s31 - m_nstar**2 + sp.I * m_nstar * gamma_nstar)
A31

In [ ]:
A31_func = sp.lambdify([s31, c0, m_nstar, gamma_nstar], A31)

### $A = A^{12}+A^{23}+A^{31}$

In [ ]:
intensity_expr = sp.Abs(A12 + A23 + A31) ** 2
intensity_expr

In [ ]:
intensity_func = sp.lambdify(
    [
        s12,
        a[-2],
        a[-1],
        a[0],
        a[1],
        a[2],
        m_a2,
        Gamma_a2,
        theta1,
        phi1,
        s23,
        b[-1],
        b[0],
        b[1],
        m_delta,
        gamma_delta,
        theta2,
        phi2,
        s31,
        c0,
        m_nstar,
        gamma_nstar,
    ],
    intensity_expr.doit().expand(func=True),
)
intensity_func

## Visualization

In [ ]:
%config InlineBackend.figure_formats = ['svg']

In [ ]:
a_vals = [0, 0.5, 3.5, 4, 2.5]
b_vals = [-1.5, 4, 0.5]
c0_val = 2.5
m_a2_val = 1.32
m_delta_val = 1.54
m_nstar_val = 1.87
Gamma_a2_val = 0.1
gamma_delta_val = 0.1
gamma_nstar_val = 0.1

In [ ]:
phi = np.pi / 4
theta = np.pi / 4
s_values = np.linspace(0, 5, num=500)
A12_values = A12_func(s_values, *a_vals, m_a2_val, Gamma_a2_val, theta, phi)
A23_values = A23_func(s_values, *b_vals, m_delta_val, gamma_delta_val, theta, phi)
A31_values = A31_func(s_values, c0_val, m_nstar_val, gamma_nstar_val)

In [ ]:
fig, ax = plt.subplots(figsize=(10, 6))
ax.plot(s_values, A12_values.imag, label="Imaginary Part", linestyle="-", color="b")
ax.plot(s_values, A12_values.real, label="Real Part", linestyle="--", color="r")
ax.plot(
    s_values,
    np.abs(A12_values) ** 2,
    label="Intensity $I^{12}=|A^{12}|^2$",
    linestyle="-.",
    color="g",
)
ax.plot(
    s_values,
    np.angle(A12_values),
    label="Phase (angle)",
    linestyle=":",
    color="m",
)
angle_projection_text = R"$\phi=\frac{\pi}{4},\theta=\frac{\pi}{4}$"
ax.set_xlabel("s")
ax.set_ylabel(r"$A^{12}$ components")
ax.set_title(Rf"Components of $A^{{12}}$ vs s ({angle_projection_text})")
ax.legend()
plt.show()

In [ ]:
fig, ax = plt.subplots()
ax.set_xlabel(R"$\mathrm{Re}\,A^{12}$")
ax.set_ylabel(R"$\mathrm{Im}\,A^{12}$")
ax.set_title(Rf"Argand diagram of $A^{{12}}$ ({angle_projection_text})")
ax.plot(A12_values.real, A12_values.imag)
plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(10, 6))
ax.plot(s_values, A23_values.imag, label="Imaginary Part", linestyle="-", color="b")
ax.plot(s_values, A23_values.real, label="Real Part", linestyle="--", color="r")
ax.plot(
    s_values,
    np.abs(A23_values) ** 2,
    label="Intensity $I=|A23|^2$",
    linestyle="-.",
    color="g",
)
ax.plot(
    s_values,
    np.angle(A23_values),
    label="Phase (angle)",
    linestyle=":",
    color="m",
)
ax.set_xlabel("s")
ax.set_ylabel(r"$A^{23}$ components")
ax.set_title(Rf"Components of $A^{{23}}$ vs s ({angle_projection_text})")
ax.legend()
plt.show()

In [ ]:
fig, ax = plt.subplots()
ax.set_xlabel(R"$\mathrm{Re}\,A^{23}$")
ax.set_ylabel(R"$\mathrm{Im}\,A^{23}$")
ax.set_title(Rf"Argand diagram of $A^{{23}}$ ({angle_projection_text})")
ax.plot(A23_values.real, A23_values.imag)
plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(10, 6))
ax.plot(s_values, A31_values.imag, label="Imaginary Part", linestyle="-", color="b")
ax.plot(s_values, A31_values.real, label="Real Part", linestyle="--", color="r")
ax.plot(
    s_values,
    np.abs(A31_values) ** 2,
    label="Intensity $I=|A31|^2$",
    linestyle="-.",
    color="g",
)
ax.plot(
    s_values,
    np.angle(A31_values),
    label="Phase (angle)",
    linestyle=":",
    color="m",
)

ax.set_xlabel("s")
ax.set_ylabel(r"$A^{31}$ components")
ax.set_title(Rf"Components of $A^{{31}}$ vs s ({angle_projection_text})")
ax.legend()
plt.show(fig)

In [ ]:
fig, ax = plt.subplots()
ax.set_xlabel(R"$\mathrm{Re}\,A^{31}$")
ax.set_ylabel(R"$\mathrm{Im}\,A^{31}$")
ax.set_title(Rf"Argand diagram of $A^{{31}}$ ({angle_projection_text})")
ax.plot(A31_values.real, A31_values.imag)
plt.show()